### Calc frames using angles

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle

def bob(max, min):
    x = max - min
    y = x / 14
    list = []
    while max > min:
        list.append(max)
        max -= y
    return list

def double_reverse_array(min, max):
    x = max - min
    y = x / 15
    list = []
    while min <= max:
        list.append(min)
        min += y
    list.append(min)
    return list

def extract_key_pushup_frames(video_path, output_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

    cap = cv2.VideoCapture(video_path)
    elbow_angles = []
    frame_images = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append((frame_count, angle))
            frame_images.append(frame)

        frame_count += 1

    cap.release()

    # Identify key frame indices
    selected_indices = []

    lowest_point = min(elbow_angles, key=lambda x: x[1])
    selected_indices.append(lowest_point[0])

    index = lowest_point[0]

    highest_point1 =  max(elbow_angles[:index], key=lambda x: x[1])
    selected_indices.append(highest_point1[0])

    highest_point2 = max(elbow_angles[index:], key=lambda x: x[1])
    selected_indices.append(highest_point2[0])

    target_angles = bob(highest_point1[1], lowest_point[1])
    target_angles2 = double_reverse_array(lowest_point[1], highest_point2[1])


    for target in target_angles:
        best_match = min(elbow_angles[:index], key=lambda x: abs(x[1] - target))
        selected_indices.append(best_match[0])

    for target in target_angles2:
        best_match = min(elbow_angles[index:], key=lambda x: abs(x[1] - target))
        selected_indices.append(best_match[0])

    selected_indices = sorted(set(selected_indices))

    # Save selected frames to output video
    height, width, _ = frame_images[0].shape
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))

    for idx in selected_indices:
        if idx < len(frame_images):
            out.write(frame_images[idx])

    out.release()
    print(f"Saved key push-up frames to {output_path}")

# Example usage
if __name__ == "__main__":
    INPUT_VIDEO = r"C:\Users\nikhi\PycharmProjects\pythonProject\fitDataa\fitDataa\Correct sequence\Copy of push up 1_flipped.mp4"
    OUTPUT_VIDEO = r"C:\Users\nikhi\PycharmProjects\pythonProject\output_keyframes.mp4"
    extract_key_pushup_frames(INPUT_VIDEO, OUTPUT_VIDEO)


KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle
"""
def bob(max, min):
    x = max - min
    y = x / 14
    list = []
    while max > min:
        list.append(max)
        max -= y
    return list

def double_reverse_array(min, max):
    x = max - min
    y = x / 15
    list = []
    while min <= max:
        list.append(min)
        min += y
    list.append(min)
    return list
"""
def extract_key_pushup_frames(video_path, output_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

    cap = cv2.VideoCapture(video_path)
    elbow_angles = []
    frame_images = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append((frame_count, angle))
            frame_images.append(frame)

        frame_count += 1

    cap.release()

    # Identify key frame indices
    selected_indices = []

    lowest_point = min(elbow_angles, key=lambda x: x[1])
    selected_indices.append(lowest_point[0])

    index = lowest_point[0]

    highest_point1 =  max(elbow_angles[:index], key=lambda x: x[1])
    selected_indices.append(highest_point1[0])

    highest_point2 = max(elbow_angles[index:], key=lambda x: x[1])
    selected_indices.append(highest_point2[0])

    """
    target_angles = bob(highest_point1[1], lowest_point[1])
    target_angles2 = double_reverse_array(lowest_point[1], highest_point2[1])

    for target in target_angles:
        best_match = min(elbow_angles[:index], key=lambda x: abs(x[1] - target))
        selected_indices.append(best_match[0])

    for target in target_angles2:
        best_match = min(elbow_angles[index:], key=lambda x: abs(x[1] - target))
        selected_indices.append(best_match[0])
    """

    selected_indices = sorted(set(selected_indices))
    print(f"Selected frame indices: {selected_indices}")
    """
    # Save selected frames to output video
    height, width, _ = frame_images[0].shape
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))

    for idx in selected_indices:
        if idx < len(frame_images):
            out.write(frame_images[idx])

    out.release()
    print(f"Saved key push-up frames to {output_path}")
    """

# Example usage
if __name__ == "__main__":
    INPUT_VIDEO = r"C:\Users\nikhi\PycharmProjects\pythonProject\fitDataa\fitDataa\Correct sequence\Copy of push up 1_flipped.mp4"
    OUTPUT_VIDEO = r"C:\Users\nikhi\PycharmProjects\pythonProject\output_keyframes.mp4"
    extract_key_pushup_frames(INPUT_VIDEO, OUTPUT_VIDEO)


In [4]:
%pip install mediapipe

  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\12484\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle

def extract_key_pushup_frames(video_path, output_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

    cap = cv2.VideoCapture(video_path)
    elbow_angles = []
    frame_images = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append((frame_count, angle))
            frame_images.append(frame)

        frame_count += 1

    cap.release()

    # Identify key frame indices
    selected_indices = []

    lowest_point = min(elbow_angles, key=lambda x: x[1])
    selected_indices.append(lowest_point[0])

    index = lowest_point[0]

    highest_point1 =  max(elbow_angles[:index], key=lambda x: x[1])
    selected_indices.append(highest_point1[0])

    highest_point2 = max(elbow_angles[index:], key=lambda x: x[1])
    selected_indices.append(highest_point2[0])

    
    selected_indices = sorted(set(selected_indices))
    print(f"Selected frame indices: {selected_indices}")
    
    # Save selected frames to output video
    height, width, _ = frame_images[0].shape
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))

    for idx in selected_indices:
        if idx < len(frame_images):
            out.write(frame_images[idx])

    out.release()
    print(f"Saved key push-up frames to {output_path}")

    

# Process Video in RepVids Directory
print("Processing videos from RepVids:")
for video_file in os.listdir(videos_path):
    if video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        video_path = os.path.join(videos_path, video_file)
        output_filename = f"key_frames_{video_file}"    # defines output format
        output_path = os.path.join(save_dir, output_filename)
        extract_key_pushup_frames(video_path, output_path)

print(f"\nAll processed videos saved to: {save_dir}")

Processing videos from RepVids:
Selected frame indices: [22, 78, 90]
Saved key push-up frames to C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\key_frames_correct_Copy of push up 100_flipped_rep_001.mp4
Selected frame indices: [24, 80, 91]
Saved key push-up frames to C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\key_frames_correct_Copy of push up 100_rep_001.mp4
Selected frame indices: [0, 31, 44]
Saved key push-up frames to C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\key_frames_correct_Copy of push up 101_flipped_rep_001.mp4
Selected frame indices: [0, 34, 45]
Saved key push-up frames to C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\key_frames_correct_Copy of push up 101_rep_001.mp4
Selected frame indices: [0, 37, 52]
Saved key push-up frames to C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\key_frames_correct_Copy of push up 102_flipped_rep_001.mp4
Selected frame indices: [0, 37, 54]
Saved key push-up frames to C:\Users\12484\Desktop\

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle

def extract_key_pushup_frames(video_path, output_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

    cap = cv2.VideoCapture(video_path)
    elbow_angles = []
    frame_images = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)
            elbow_angles.append((frame_count, angle))
            frame_images.append(frame)

        frame_count += 1

    cap.release()

    # Identify key frame indices
    selected_indices = []

    lowest_point = min(elbow_angles, key=lambda x: x[1])
    selected_indices.append(lowest_point[0])

    index = lowest_point[0]

    highest_point1 =  max(elbow_angles[:index], key=lambda x: x[1])
    selected_indices.append(highest_point1[0])

    highest_point2 = max(elbow_angles[index:], key=lambda x: x[1])
    selected_indices.append(highest_point2[0])

    
    selected_indices = sorted(set(selected_indices))
    print(f"Selected frame indices: {selected_indices}")
    
    # Save selected frames to output video
    height, width, _ = frame_images[0].shape
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))

    for idx in selected_indices:
        if idx < len(frame_images):
            out.write(frame_images[idx])

    out.release()
    print(f"Saved key push-up frames to {output_path}")

def downsample_to_30_frames(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    
    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    print(f"Input: {total_frames} frames")
    
    # Target exactly 30 frames
    target_frames = 30
    
    if total_frames <= target_frames:
        selected_indices = list(range(total_frames))
    else:
        selected_indices = np.linspace(0, total_frames - 1, target_frames, dtype=int)
    
    print(f"Selected indices: {selected_indices}")
    
    # Create output video writer
    out = cv2.VideoWriter(output_path, fourcc, 10.0, (width, height))
    
    # Extract selected frames
    for i, frame_idx in enumerate(selected_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        
        if ret:
            out.write(frame)
        else:
            print(f"❌ Failed to read frame {frame_idx}")
    
    cap.release()
    out.release()
    print(f"✅ Output: {len(selected_indices)} frames saved")

# Process all videos in RepKeys directory to exactly 30 frames
def process_repkeys_to_30_frames():
    # Input and output directories
    input_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepVids")
    output_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepKeys")
    
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"🎬 Processing videos from: {input_dir}")
    print(f"💾 Saving 30-frame videos to: {output_dir}")
    
    # Process each video file
    for video_file in os.listdir(input_dir):
        if video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            input_path = os.path.join(input_dir, video_file)
            output_filename = f"30frames_{video_file}"
            output_path = os.path.join(output_dir, output_filename)
            
            print(f"\n🔄 Processing: {video_file}")
            downsample_to_30_frames(input_path, output_path)
    
    print(f"\n🎉 All videos processed and saved to: {output_dir}")

# Run the processing
process_repkeys_to_30_frames()

In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle

def extract_key_frames_and_downsample_to_30(input_path, output_path):
    """
    Combined function: Extract key pushup frames THEN downsample to exactly 30 frames
    """
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

    cap = cv2.VideoCapture(input_path)
    elbow_angles = []
    frame_images = []
    frame_count = 0

    print(f"🔍 Analyzing frames for key points...")
    
    # STEP 1: Extract frames with pose landmarks and calculate angles
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            
            # Try both sides - use whichever is more visible
            try:
                # Try LEFT side first
                left_shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                left_visibility = (lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility + 
                                 lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility + 
                                 lm[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility) / 3
                
                # Try RIGHT side
                right_shoulder = [lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                right_visibility = (lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility + 
                                  lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility + 
                                  lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility) / 3
                
                # Use the side with better visibility
                if left_visibility >= right_visibility and left_visibility > 0.5:
                    shoulder, elbow, wrist = left_shoulder, left_elbow, left_wrist
                elif right_visibility > 0.5:
                    shoulder, elbow, wrist = right_shoulder, right_elbow, right_wrist
                else:
                    frame_count += 1
                    continue
                
                angle = calculate_angle(shoulder, elbow, wrist)
                elbow_angles.append((frame_count, angle))
                frame_images.append(frame)
                
            except:
                pass

        frame_count += 1

    cap.release()
    
    if len(elbow_angles) < 3:
        print(f"❌ Not enough frames with pose landmarks ({len(elbow_angles)}). Skipping...")
        return False

    # STEP 2: Identify key frame indices based on elbow angles
    print(f"📊 Found {len(elbow_angles)} frames with pose data")
    
    selected_indices = []

    # Find lowest point (most bent elbow)
    lowest_point = min(elbow_angles, key=lambda x: x[1])
    selected_indices.append(lowest_point[0])
    index = lowest_point[0]

    # Find highest points before and after lowest point
    if index > 0:
        highest_point1 = max(elbow_angles[:index], key=lambda x: x[1])
        selected_indices.append(highest_point1[0])

    if index < len(elbow_angles) - 1:
        highest_point2 = max(elbow_angles[index:], key=lambda x: x[1])
        selected_indices.append(highest_point2[0])

    # Remove duplicates and sort
    selected_indices = sorted(set(selected_indices))
    print(f"🎯 Key frame indices: {selected_indices}")
    
    # STEP 3: Extract key frames
    key_frames = []
    for idx in selected_indices:
        if idx < len(frame_images):
            key_frames.append(frame_images[idx])
    
    if len(key_frames) == 0:
        print(f"❌ No key frames extracted. Skipping...")
        return False
    
    print(f"🔑 Extracted {len(key_frames)} key frames")
    
    # STEP 4: Downsample to exactly 30 frames
    target_frames = 30
    
    if len(key_frames) <= target_frames:
        # If we have 30 or fewer key frames, use all of them
        final_frames = key_frames
        print(f"📦 Using all {len(key_frames)} key frames (≤30)")
    else:
        # If we have more than 30, downsample evenly
        selected_frame_indices = np.linspace(0, len(key_frames) - 1, target_frames, dtype=int)
        final_frames = [key_frames[i] for i in selected_frame_indices]
        print(f"⬇️ Downsampled from {len(key_frames)} to {len(final_frames)} frames")
    
    # STEP 5: Save output video
    if len(final_frames) > 0:
        height, width, _ = final_frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, 10.0, (width, height))
        
        for frame in final_frames:
            out.write(frame)
        
        out.release()
        print(f"✅ Saved {len(final_frames)} frames to: {output_path}")
        return True
    else:
        print(f"❌ No frames to save")
        return False

# Process all videos in RepVids directory
def process_repvids_to_repkeys():
    """
    Process all videos in RepVids: Extract key frames → Downsample to 30 frames → Save to RepKeys
    """
    # Input and output directories
    input_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepVids")
    output_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepKeys")
    
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"🎬 Processing videos from: {input_dir}")
    print(f"💾 Saving processed videos to: {output_dir}")
    print(f"🎯 Target: Key frames → 30 frames max\n")
    
    # Get all video files
    video_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    
    if not video_files:
        print("❌ No video files found in RepVids directory!")
        return
    
    successful = 0
    failed = 0
    
    # Process each video file
    for i, video_file in enumerate(video_files, 1):
        input_path = os.path.join(input_dir, video_file)
        output_filename = f"keyframes_30_{video_file}"
        output_path = os.path.join(output_dir, output_filename)
        
        print(f"[{i}/{len(video_files)}] 🔄 Processing: {video_file}")
        
        try:
            success = extract_key_frames_and_downsample_to_30(input_path, output_path)
            if success:
                successful += 1
                print(f"✅ Success!\n")
            else:
                failed += 1
                print(f"❌ Failed!\n")
        except Exception as e:
            failed += 1
            print(f"❌ Error: {e}\n")
    
    print("="*60)
    print(f"🎉 PROCESSING COMPLETE!")
    print(f"✅ Successful: {successful}")
    print(f"❌ Failed: {failed}")
    print(f"📁 Output directory: {output_dir}")
    print("="*60)

    # Return the counts so they can be accessed outside the function
    return successful, failed

# Run the processing
if __name__ == "__main__":
    successful, failed = process_repvids_to_repkeys()

🎬 Processing videos from: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepVids
💾 Saving processed videos to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys
🎯 Target: Key frames → 30 frames max

[1/187] 🔄 Processing: correct_Copy of push up 100_flipped_rep_001.mp4
🔍 Analyzing frames for key points...
📊 Found 91 frames with pose data
🎯 Key frame indices: [22, 78, 90]
🔑 Extracted 3 key frames
📦 Using all 3 key frames (≤30)
✅ Saved 3 frames to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\keyframes_30_correct_Copy of push up 100_flipped_rep_001.mp4
✅ Success!

[2/187] 🔄 Processing: correct_Copy of push up 100_rep_001.mp4
🔍 Analyzing frames for key points...
📊 Found 92 frames with pose data
🎯 Key frame indices: [0, 79, 91]
🔑 Extracted 3 key frames
📦 Using all 3 key frames (≤30)
✅ Saved 3 frames to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\keyframes_30_correct_Copy of push up 100_rep_001.mp4
✅ Success!

[3/187] 🔄 Processing: correct_Copy of push up 101_f

KeyboardInterrupt: 

### *Downsampling to 30

In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle

def extract_key_frames_and_downsample_to_30(input_path, output_path):
    """
    Extract frames between key points and downsample to 30 frames with guaranteed minimum elbow angle
    """
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

    cap = cv2.VideoCapture(input_path)
    elbow_angles = []
    frame_count = 0

    print(f"Analyzing frames for key points...")
    
    # STEP 1: First pass - extract frames and calculate angles
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            
            # Try both sides - use whichever is more visible
            try:
                # Try LEFT side first
                left_shoulder = [lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                left_visibility = (lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility + 
                                 lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility + 
                                 lm[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility) / 3
                
                # Try RIGHT side
                right_shoulder = [lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                right_visibility = (lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility + 
                                  lm[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility + 
                                  lm[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility) / 3
                
                # Use the side with better visibility
                if left_visibility >= right_visibility and left_visibility > 0.5:
                    shoulder, elbow, wrist = left_shoulder, left_elbow, left_wrist
                elif right_visibility > 0.5:
                    shoulder, elbow, wrist = right_shoulder, right_elbow, right_wrist
                else:
                    frame_count += 1
                    continue
                
                angle = calculate_angle(shoulder, elbow, wrist)
                elbow_angles.append((frame_count, angle))
                
            except:
                pass

        frame_count += 1

    cap.release()
    
    if len(elbow_angles) < 3:
        print(f"Not enough frames with pose landmarks ({len(elbow_angles)}). Skipping...")
        return False

    # STEP 2: Identify key frame indices based on elbow angles
    print(f"Found {len(elbow_angles)} frames with pose data")
    
    # Find lowest point (most bent elbow) - MUST be included
    lowest_point = min(elbow_angles, key=lambda x: x[1])
    lowest_frame_index = lowest_point[0]
    
    print(f"Minimum elbow angle: {lowest_point[1]:.1f}° at frame {lowest_frame_index}")

    # Find highest points before and after lowest point
    key_indices = []
    
    if lowest_frame_index > 0:
        # Find frames with pose data before the lowest point
        before_frames = [(idx, angle) for idx, angle in elbow_angles if idx < lowest_frame_index]
        if before_frames:
            highest_before = max(before_frames, key=lambda x: x[1])
            start_frame = highest_before[0]
            print(f"Start frame: {start_frame} (angle: {highest_before[1]:.1f}°)")
        else:
            start_frame = 0
    else:
        start_frame = 0

    if lowest_frame_index < frame_count - 1:
        # Find frames with pose data after the lowest point
        after_frames = [(idx, angle) for idx, angle in elbow_angles if idx > lowest_frame_index]
        if after_frames:
            highest_after = max(after_frames, key=lambda x: x[1])
            end_frame = highest_after[0]
            print(f"End frame: {end_frame} (angle: {highest_after[1]:.1f}°)")
        else:
            end_frame = frame_count - 1
    else:
        end_frame = frame_count - 1

    print(f"Extracting frames from {start_frame} to {end_frame} (total: {end_frame - start_frame + 1} frames)")
    
    # STEP 3: Extract all frames between key points
    cap = cv2.VideoCapture(input_path)
    all_frames_in_range = []
    frame_indices_in_range = []
    
    current_frame = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        if start_frame <= current_frame <= end_frame:
            all_frames_in_range.append(frame)
            frame_indices_in_range.append(current_frame)
            
        current_frame += 1
        
        # Stop if we've passed the end frame
        if current_frame > end_frame:
            break

    cap.release()
    
    if len(all_frames_in_range) == 0:
        print(f"No frames extracted in range. Skipping...")
        return False
    
    print(f"Extracted {len(all_frames_in_range)} frames between key points")
    
    # STEP 4: Downsample to exactly 30 frames while ensuring minimum angle frame is included
    target_frames = 30
    
    if len(all_frames_in_range) <= target_frames:
        # If we have 30 or fewer frames, use all of them
        final_frames = all_frames_in_range
        final_indices = frame_indices_in_range
        print(f"Using all {len(all_frames_in_range)} frames (≤30)")
    else:
        # Downsample but GUARANTEE the minimum elbow angle frame is included
        
        # Find the position of the minimum elbow angle frame in our extracted range
        min_frame_position_in_range = lowest_frame_index - start_frame
        
        # Create evenly spaced indices
        selected_positions = np.linspace(0, len(all_frames_in_range) - 1, target_frames, dtype=int)
        
        # Ensure the minimum elbow angle frame is included
        if min_frame_position_in_range not in selected_positions:
            # Replace the closest selected position with the minimum angle frame position
            closest_idx = np.argmin(np.abs(selected_positions - min_frame_position_in_range))
            selected_positions[closest_idx] = min_frame_position_in_range
            selected_positions = np.sort(selected_positions)
        
        final_frames = [all_frames_in_range[i] for i in selected_positions]
        final_indices = [frame_indices_in_range[i] for i in selected_positions]
        
        print(f"⬇Downsampled from {len(all_frames_in_range)} to {len(final_frames)} frames")
        print(f"Minimum elbow angle frame {lowest_frame_index} is guaranteed to be included")
    
    # STEP 5: Save output video
    if len(final_frames) > 0:
        height, width, _ = final_frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, 10.0, (width, height))
        
        for frame in final_frames:
            out.write(frame)
        
        out.release()
        print(f"Saved {len(final_frames)} frames to: {output_path}")
        return True
    else:
        print(f"No frames to save")
        return False

# Process all videos in RepVids directory
def process_repvids_to_repkeys():
    """
    Process all videos in RepVids: Extract frames between key points → Downsample to 30 frames → Save to RepKeys
    """
    # Input and output directories
    input_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepVids")
    output_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepKeys")
    
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Processing videos from: {input_dir}")
    print(f"Saving processed videos to: {output_dir}")
    print(f"Target: Extract frames between key points → 30 frames max with guaranteed minimum elbow angle\n")
    
    # Get all video files
    video_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    
    if not video_files:
        print("No video files found in RepVids directory!")
        return 0, 0
    
    successful = 0
    failed = 0
    
    # Process each video file
    for i, video_file in enumerate(video_files, 1):
        input_path = os.path.join(input_dir, video_file)
        output_filename = f"keyframes_30_{video_file}"
        output_path = os.path.join(output_dir, output_filename)
        
        print(f"[{i}/{len(video_files)}] Processing: {video_file}")
        
        try:
            success = extract_key_frames_and_downsample_to_30(input_path, output_path)
            if success:
                successful += 1
                print(f"Success!\n")
            else:
                failed += 1
                print(f"Failed!\n")
        except Exception as e:
            failed += 1
            print(f"Error: {e}\n")
    
    print("="*60)
    print(f"PROCESSING COMPLETE!")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Output directory: {output_dir}")
    print("="*60)

    # Return the counts so they can be accessed outside the function
    return successful, failed

# Run the processing
if __name__ == "__main__":
    successful, failed = process_repvids_to_repkeys()

Processing videos from: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepVids
Saving processed videos to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys
Target: Extract frames between key points → 30 frames max with guaranteed minimum elbow angle

[1/182] Processing: correct_Copy of push up 100_flipped_rep_001.mp4
Analyzing frames for key points...
Found 97 frames with pose data
Minimum elbow angle: 63.7° at frame 78
Start frame: 22 (angle: 177.2°)
End frame: 96 (angle: 157.3°)
Extracting frames from 22 to 96 (total: 75 frames)
Extracted 75 frames between key points
⬇Downsampled from 75 to 30 frames
Minimum elbow angle frame 78 is guaranteed to be included
Saved 30 frames to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepKeys\keyframes_30_correct_Copy of push up 100_flipped_rep_001.mp4
Success!

[2/182] Processing: correct_Copy of push up 100_rep_001.mp4
Analyzing frames for key points...
Found 99 frames with pose data
Minimum elbow angle: 64.6° at frame 79
Start frame:

In [2]:
print(f"Successful videos: {successful}")
print(f"Failed videos: {failed}")
print(f"Success rate: {(successful/(successful+failed)*100):.1f}%" if (successful+failed) > 0 else "No videos processed")

Successful videos: 182
Failed videos: 0
Success rate: 100.0%
